In [ ]:
# Import libraries
import pandas as pd
import numpy as np

In [ ]:
# Load housing data
# Load specific columns including 'MEASURE' to filter correctly
housing_cols = ['Reference area', 'TIME_PERIOD', 'OBS_VALUE', 'FREQ', 'MEASURE']
df_prices = pd.read_csv('housing_prices_raw.csv', usecols=housing_cols)

In [ ]:
# Filter: Quarterly Data (Q) AND Real House Price Index (RHPI)
# This filter is crucial to avoid mixing different data types and prevents errors.
df_prices = df_prices[(df_prices['FREQ'] == 'Q') & (df_prices['MEASURE'] == 'RHPI')].copy()
df_prices.head()

,Reference area,FREQ,MEASURE,TIME_PERIOD,OBS_VALUE
65554,Capital region,Q,RHPI,1992-Q2,0.000
65555,Capital region,Q,RHPI,1992-Q3,-2.454
65556,Capital region,Q,RHPI,1992-Q4,-3.145
65557,Capital region,Q,RHPI,1993-Q1,-1.299
65558,Capital region,Q,RHPI,1993-Q2,0.000


In [ ]:
# Rename columns
country_fix_map = {
    'Czech Republic': 'Czechia',
    'Türkiye': 'Turkey',
    'Korea': 'Korea',
    'Reference area': 'Country'
}
df_prices['Reference area'] = df_prices['Reference area'].replace(country_fix_map)
df_prices.rename(columns={'Reference area': 'Country', 'TIME_PERIOD': 'Time', 'OBS_VALUE': 'House Price Index'}, inplace=True)
df_prices.head()

,Country,FREQ,MEASURE,Time,House Price Index
65554,Capital region,Q,RHPI,1992-Q2,0.000
65555,Capital region,Q,RHPI,1992-Q3,-2.454
65556,Capital region,Q,RHPI,1992-Q4,-3.145
65557,Capital region,Q,RHPI,1993-Q1,-1.299
65558,Capital region,Q,RHPI,1993-Q2,0.000


In [ ]:
# Define Major Economies (Valid Countries)
valid_countries = [
    'Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 'Colombia',
    'Costa Rica', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France',
    'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Israel',
    'Italy', 'Japan', 'Korea', 'Latvia', 'Lithuania', 'Luxembourg',
    'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
    'Portugal', 'Slovak Republic', 'Slovenia', 'Spain', 'Sweden',
    'Switzerland', 'Turkey', 'United Kingdom', 'United States'
]

# Keep only valid countries
df_prices = df_prices[df_prices['Country'].isin(valid_countries)]
# Take index, remove change ratio
df_prices = df_prices.groupby(['Country', 'Time'], as_index=False)['House Price Index'].max()
df_prices.head()

,Country,Time,House Price Index
0,Australia,2002-Q1,100.000000
1,Australia,2002-Q2,105.948489
2,Australia,2002-Q3,110.364042
3,Australia,2002-Q4,114.395740
4,Australia,2003-Q1,117.466568


In [ ]:
# Normalize to Base 100
# Ensures every country starts at 100 for a fair comparison with Income Data.
# Formula: (Value / First_Value) * 100
df_prices.sort_values(['Country', 'Time'], inplace=True)
df_prices['House Price Index'] = df_prices.groupby('Country')['House Price Index'].transform(lambda x: (x / x.iloc[0]) * 100)
df_prices.head()

,Country,Time,House Price Index
0,Australia,2002-Q1,100.000000
1,Australia,2002-Q2,105.948489
2,Australia,2002-Q3,110.364042
3,Australia,2002-Q4,114.395740
4,Australia,2003-Q1,117.466568


In [ ]:
# Load income data
df_income_raw = pd.read_csv('household_income.csv')
df_income_raw.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,FREQ,Frequency of observation,REF_AREA,Reference area,MEASURE,Measure,...,CONF_STATUS,Confidentiality status,DECIMALS,Decimals,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,CURRENCY,Currency
0,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,AUS,Australia,LES1M_F2AS,Share of currency and deposits in financial as...,...,F,Free (free for publication),2,Two,A,Normal value,0,Units,_Z,Not applicable
1,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,AUT,Austria,LES1M_F2AS,Share of currency and deposits in financial as...,...,F,Free (free for publication),2,Two,A,Normal value,0,Units,_Z,Not applicable
2,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,BEL,Belgium,LES1M_F2AS,Share of currency and deposits in financial as...,...,F,Free (free for publication),2,Two,A,Normal value,0,Units,_Z,Not applicable
3,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,CAN,Canada,LES1M_F2AS,Share of currency and deposits in financial as...,...,F,Free (free for publication),2,Two,A,Normal value,0,Units,_Z,Not applicable
4,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,CZE,Czechia,LES1M_F2AS,Share of currency and deposits in financial as...,...,F,Free (free for publication),2,Two,A,Normal value,0,Units,_Z,Not applicable


In [ ]:
# Filter for the specific income measure
# Code: B7N_R_PPP_P41 -> Real Net Adjusted Disposable Income
df_income = df_income_raw[df_income_raw['MEASURE'] == 'B7N_R_PPP_P41'].copy()
df_income.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,FREQ,Frequency of observation,REF_AREA,Reference area,MEASURE,Measure,...,CONF_STATUS,Confidentiality status,DECIMALS,Decimals,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,CURRENCY,Currency
11015,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EA,Euro area,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable
11017,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EU,European Union,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable
11019,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EA,Euro area,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable
11021,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EU,European Union,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable
11022,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EA,Euro area,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable


In [ ]:
#Rename Columns
new_names_income = {
    'Reference area': 'Country',
    'TIME_PERIOD': 'Year',
    'OBS_VALUE': 'Income_Value'}
df_income.rename(columns=new_names_income, inplace=True)
df_income.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,FREQ,Frequency of observation,REF_AREA,Country,MEASURE,Measure,...,CONF_STATUS,Confidentiality status,DECIMALS,Decimals,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,CURRENCY,Currency
11015,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EA,Euro area,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable
11017,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EU,European Union,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable
11019,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EA,Euro area,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable
11021,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EU,European Union,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable
11022,DATAFLOW,OECD.SDD.NAD:DSD_NAAG@DF_NAAG_V(1.0),NAAG Chapter 5: Households,I,A,Annual,EA,Euro area,B7N_R_PPP_P41,Real net adjusted disposable income of househo...,...,F,Free (free for publication),2,Two,A,Normal value,6,Millions,_Z,Not applicable


In [ ]:
# Transform Annual to Quarterly

df_income['Date'] = pd.to_datetime(df_income['Year'].astype(str) + '-01-01')
df_income.set_index('Date', inplace=True)
df_income_quarterly = df_income.groupby('Country')['Income_Value'].resample('QE').ffill().reset_index()

In [ ]:
# Normalize to base 100
# This converts "Million Dollars" into a "100-point Index" to match housing data
df_income_quarterly['Income Index'] = df_income_quarterly.groupby('Country')['Income_Value'].transform(lambda x: (x / x.iloc[0]) * 100)

In [ ]:
# Format Time (YYYY-Qx)
df_income_quarterly['Time'] = df_income_quarterly['Date'].dt.to_period('Q').astype(str).str.replace('Q', '-Q')
df_income_quarterly.head()

,Country,Date,Income_Value,Income Index,Time
0,Australia,1970-03-31,314775.009222,100.000000,1970-Q1
1,Australia,1970-06-30,314775.009222,100.000000,1970-Q2
2,Australia,1970-09-30,314775.009222,100.000000,1970-Q3
3,Australia,1970-12-31,314775.009222,100.000000,1970-Q4
4,Australia,1971-03-31,328089.969507,104.229993,1971-Q1


In [ ]:
# Merge DataFrames
df_final = pd.merge(df_prices,
                    df_income_quarterly[['Country', 'Time', 'Income Index']],
                    on=['Country', 'Time'])
#Rebase to 100
# Reset both Housing and Income indices to start exactly at 100.0 for the first common date.
df_final.sort_values(['Country', 'Time'], inplace=True)

# Rebase House Price Index to start at 100
df_final['House Price Index'] = df_final.groupby('Country')['House Price Index'].transform(lambda x: (x / x.iloc[0]) * 100)

# Rebase Income Index to start at 100
df_final['Income Index'] = df_final.groupby('Country')['Income Index'].transform(lambda x: (x / x.iloc[0]) * 100)

In [ ]:
# Calculate Affordability Ratio
df_final['Affordability Ratio'] = (df_final['House Price Index'] / df_final['Income Index']) * 100
df_final['Affordability Ratio'] = df_final['Affordability Ratio'].round(2)

In [ ]:
# Select only needed columns
final_columns = ['Country', 'Time', 'House Price Index', 'Income Index', 'Affordability Ratio']
df_final = df_final[final_columns]

In [ ]:
# Check last version
df_final.head()

,Country,Time,House Price Index,Income Index,Affordability Ratio
0,Australia,2002-Q1,100.000000,100.00000,100.00
1,Australia,2002-Q2,105.948489,100.00000,105.95
2,Australia,2002-Q3,110.364042,100.00000,110.36
3,Australia,2002-Q4,114.395740,100.00000,114.40
4,Australia,2003-Q1,117.466568,105.95585,110.86


In [ ]:
#Convert to CSV
df_final.to_csv('Housing Affordability.csv', index=False)